In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(dplyr)
library(randomForest)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin



In [2]:
set.seed(1)

In [3]:
jRoadType <- c("都市間高速", "都市高速", "有料道路", "国道", "県道", "主要地方道", "一般道1、一般道2、一般道3", "その他")
jHighSpeeds <- c("都市間高速", "都市高速", "有料道路")

In [4]:
valuables <- c("RoadType","CurveAverage","Speed", "Curve100", "Curve150","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed", "Engine", "SteeringAngle", "TimeHeadway", "Jerk", "LaneCount", "DiffAvgSpeed", "EmptinessOfRoad", "RoadFactor")

In [5]:
printf <- function(...) cat(sprintf(...))

In [6]:
trainAndPredict <- function(dfx, verbose=FALSE) {        
    train <- dfx[, c(valuables, "flag")]
    test <- dfx
    fit <- randomForest(flag ~ ., data=train, mtry=3, ntree=500)
    p <- predict(fit, newdata=test)

    showMatrix(test, p)

    nerr <- sum(p != test$flag)
}

In [7]:
showMatrix <- function(test, predictedFlags) {
    predictedRed <- test[predictedFlags == "Red", ]
    predictedBlue <- test[predictedFlags == "Blue", ]
    print(predictedRed %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    print(predictedBlue %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
}

# Predict Reds

In [8]:
df3 <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [9]:
df3$AccelOrBrake[df3$flag == "RedA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "RedB"] <- "Brake"
df3$AccelOrBrake[df3$flag == "BlueA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "BlueB"] <- "Brake"

In [10]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [11]:
allFeatures <- c(colnames(df3))

In [12]:
roadTypes <- unique(df3$RoadType)

In [13]:
df <- df3[, c(valuables, "AccelOrBrake", "flag")]

In [14]:
for (i in roadTypes) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df[df$RoadType == i, ]
    print("=============TRUE================")
    print(dfx %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    print("=============TEST================")
    result <- trainAndPredict(dfx, verbose=TRUE)
}

RoadType: 7 (その他)
[1] "=============TRUE================"
Source: local data frame [3 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Brake     2
2    Red        Accel     5
3    Red        Brake    20
[1] "=============TEST================"
Source: local data frame [2 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1    Red        Accel     5
2    Red        Brake    20
Source: local data frame [1 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Brake     2
RoadType: 6 (一般道1、一般道2、一般道3)
[1] "=============TRUE================"
Source: local data frame [4 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fctr>        <chr> <int>
1   Blue        Accel     7
2   Blue        Brake    92
3    Red        Accel    50
4    Red        Brake    40
[1] "=============TEST================"
Source: local data frame [2 x 3]
Groups: flag [?]

    flag AccelOrBrake count
  <fct